## Применение модели NER-BERT, дообученной на движке torch

В тетрадке 2 основных блока:

    1. Получение тегов по одной введенной строке - позволяет ввести свою строку для анализа на наличие тегов
    2. Массовое тестирование модели на 20 000 примерах - позволяет сравнить ответы трех моделей: LD, DeepPavlov, Natasha

In [2]:
import torch
from pathlib import Path
import sys
sys.path.append("..")
from src import NerTokes, Ner, NerPipe
from src.ner_statistic import nomalize_spans, get_stat

import pandas as pd
from IPython.display import display, HTML

from datasets import load_dataset, load_metric

import shutil
import os
from transformers import AutoModelForTokenClassification, AutoTokenizer
from deeppavlov.models.tokenizers.nltk_moses_tokenizer import NLTKMosesTokenizer

from ipymarkup import show_box_markup
from ipymarkup.palette import *

import ast
import operator

BASE_DIR = Path().cwd().parent
wikiann_dl_dir = BASE_DIR.joinpath("data/interim")

### Поднимаем модельку с диска

In [2]:
import transformers
torch.__version__

'1.7.0+cu92'

In [3]:
# относительное расположение модели
# model_path = 'models/ner_dpbert_100epoch_88f1_95acc.bin'
model_path = 'models/ner_dpbert_3epoch_95f1ORG_99acc.bin'
# model_path = 'models/ner_dpbert_4epoch_99f1ORG_99acc.bin'
# model_path = 'models/ner_dpbert_7epoch_56f1ORG_96acc.bin'

# объединяем путь к корневой директории с относительным расположением модели
model_checkpoint = BASE_DIR.joinpath(model_path)

label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-INNKPP', 'I-INNKPP', 'B-RSKS', 'I-RSKS', 'B-STAT', 'I-STAT']
nlp_ner = NerPipe(model_checkpoint, label_list, '##',0.8) #Ner(model_checkpoint, label_list)

#  Функция для получения текста от пользователя.
def get_user_text():
    """
    Функция для получения введеной пользователем строки текста.
    """
    while True:
        text = input("Введите текст:")
        break
    return text

### Получение тегов по одной введенной строке

Для тестирования необходимо запустить ячейку, далее будет предложено ввести строку и система сразу произведет ее анализ

<b>Нер работает для строк из первичных документов, на строках свободного написания сильно ошибается</b>

In [22]:
text = get_user_text()

nlp_ner.get_spans(text)
# nlp_ner.get_span_coordinates()

show_box_markup(text, nlp_ner.spans_coord, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))

Введите текст:получатель ООО "ТД" ЭЛЕКОМ " адресу 197374, Санкт-Петербург город улица Стародеревенская дом 11 корпус 2 ЛИТЕР А ЭТАЖ 4 ОФИС 429 ИННКПП получатель 1814740712 / 781401001


In [6]:
text = get_user_text()
nlp_ner.get_spans(text)
# nlp_ner.get_span_coordinates()

show_box_markup(text, nlp_ner.spans_coord, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))

Введите текст:Исполнитель Индивидуальный предприниматель Ефимова Наталья Ивановна, +8-826-568-7012 Адрес банка Исп-тель 190098 , г Санкт-Петербург , Адмиралтейский р-н , пл Труда , д 4 литер в , пом 212 231111056 Небанковская кредитная организация " МОНЕТА " ( общество с ограниченной ответственностью ) рас/сч 40253440786260473970 Адрес банка 614015 , Пермский край , г Пермь , Ленинский р-н , ул Екатерининская , д 18 тел. 8-885-63-06-595 кор/сч 30647910558312445795 БИК 046792177


In [16]:
nlp_ner.pipe_res

[{'word': 'ром',
  'score': 0.7372082471847534,
  'entity': 'LABEL_4',
  'index': 1,
  'start': 0,
  'end': 3},
 {'word': '##ашка',
  'score': 0.8091588020324707,
  'entity': 'LABEL_4',
  'index': 2,
  'start': 3,
  'end': 7},
 {'word': '-',
  'score': 0.5067232847213745,
  'entity': 'LABEL_0',
  'index': 3,
  'start': 7,
  'end': 8},
 {'word': 'профиль',
  'score': 0.8298013806343079,
  'entity': 'LABEL_4',
  'index': 4,
  'start': 8,
  'end': 15},
 {'word': ',',
  'score': 0.8108944296836853,
  'entity': 'LABEL_4',
  'index': 5,
  'start': 15,
  'end': 16},
 {'word': 'о',
  'score': 0.7483734488487244,
  'entity': 'LABEL_4',
  'index': 6,
  'start': 17,
  'end': 18},
 {'word': '##оо',
  'score': 0.6099687814712524,
  'entity': 'LABEL_3',
  'index': 7,
  'start': 18,
  'end': 20},
 {'word': 'о',
  'score': 0.8081063628196716,
  'entity': 'LABEL_4',
  'index': 8,
  'start': 22,
  'end': 23},
 {'word': '##оо',
  'score': 0.7045565843582153,
  'entity': 'LABEL_4',
  'index': 9,
  'start'

In [6]:
texts = [
        'рес 111394 город Москва проспект ЗЕЛЁНЫЙ 34 ЦОКОЛЬ ПОМ.1 - КОМ.4В',
        'Ромашка-профиль, ООО  кор. счет 30101810300000000709 9204000492/92040001  "ЛИТЛТОН",ООО 299011 '\
        'г. Севастополь ул. Адмирала Октябрьского д. 13 офис 1 тел (0692) 671710 ‚  в '\
        'Банк вфыSSF d АО "ТААТТА", г. Иркутск БИК 049805709 40702810500000007563   по ОКПО',
        'валюта наименование код Российская рубль 643',
        'Инвестор ООО ДЮКА ИНН КПП 564500394 / 231201001 624134, Свердловская обл, г Новоуральск, ул Чурина, д 12/2, оф 20 рс 44468253734659494408 Филиал банка П-щик АКЦИОНЕРНОЕ ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК "КУБАНСКИЙ ТОРГОВЫЙ БАНК" ',
        'СЕВЕРНОЕ ЛИНЕЙНОЕ ПРОИЗВОДСТВЕННОЕ УПРАВЛЕНИЕ МАГИСТРАЛЬНЫХ ГАЗОПРОВОДОВ (СЕВЕРНОЕ ЛПУМГ) 631226435882 ГОРОД МОСКВА УЛИЦА УСАЧЁВА ДОМ 24 Филиал банка Плательщик ООО Банк "Саратов" тел. 8 344 741 9083 БИК ',
        'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "МАКТЕЛ" ИНН П-щик 6330058874',
        'исполнитель МКУ ЕДДС Кемеровская область, город Кемерово счет № 43209075037153306310 банк ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО "ВОСТОККРЕДИТБАНК" к/с 30732437717823862544 тел. +7 (820) 529 92 19',
        'ООО АНТ-СОФТ ИНН/КПП 3911800906 / 431201001 Банк ИНН КПП 5519504954 / тел. +8-( 873) -958-9191 БИК 042306734',
        'З-чик АО ПРОИЗВОДСТВЕННАЯ КОМПАНИЯ СТАНКОПРЕСС ИНН Покупатель 770856524 ОБЛАСТЬ АМУРСКАЯ ГОРОД БЛАГОВЕЩЕНСК УЛИЦА ЛЕНИНА 181 Банк Подрядчик КБ "Русский ипотечный банк" (ООО) тел. +7 (982) 099 40 08 045402896',
        'МУП МП ГОРОДСКОГО ОКРУГА САМАРА КРАСНОГЛИНСКИЕ БАНИ +7-830-199-8031 Адрес филиала банка Брянская область, город Новозыбков Банк Инвестор ИНН/КПП 100601303468 / 632501001 корр. счет 30310411361955797442',
        ]

### Запуск по одному из текстов массива texts

Ниже можно посмотреть как модель отрабатывает точечно на отдельных тестах
1. Для запуска работы необходимо проинициализировать массив примеров в ячейке выше и далее в ячейке ниже выбрать по номеру один из них (нумерация начинается с 0)

2. Если есть желание изменить содержание какой-либо строки, это можноделать прямо в ячейке выше (незабываем перезапускать ечейку после каждого измменения)

3. Если хочется придумать полностью свой текст, то его необходимо присвоить переменной текст в ячейке ниже<br>
<br>
<b>EX:</b> text = 'Заказчик АО Газпром, 12345789/87000987'

In [7]:
text = texts[2]
nlp_ner.get_spans(text)
# nlp_ner.get_span_coordinates()

show_box_markup(text, nlp_ner.spans_coord, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))

In [7]:
text = 'получатель ООО "ГАРАНТ-СВ” А ”5№. heOFVS1 Адрес 298685. РФ респ. Крым гор. Ялта ул Генерала О оякова. д. 9 -Р ви ИННКПП получатель 9103007830/910301001'
nlp_ner.get_spans(text)
# nlp_ner.get_span_coordinates()

show_box_markup(text, nlp_ner.spans_coord, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))

In [11]:
text = 'акт 112 от 11 июля 2011 года о приемке выполненных работ (оказанных услуг)'
nlp_ner.get_spans(text)
# nlp_ner.get_span_coordinates()

show_box_markup(text, nlp_ner.spans_coord, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))

### Массовый запуск по всему массиву texts

In [12]:
for i, text in enumerate(texts):
    _ = nlp_ner.get_spans(text)
#     nlp_ner.get_span_coordinates()
    print(f'ex {i}:')
    show_box_markup(text, nlp_ner.spans_coord, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))

ex 0:


ex 1:


ex 2:


ex 3:


ex 4:


ex 5:


ex 6:


ex 7:


ex 8:


ex 9:


## Массовое тестирование модели на 20 000 примерах

Запуск массового тестирования проиводился заранее.<br>
Результаты тестирования всех моделей: LD_NER, DEEPPAVLOV, NATASHA занесены в файл NER_TEST_LD.xlsx<br>
Чтобы посмотреть результаты по каждой из моделей необходимо подгрузить файлв Dataframe

Подгружаем данные для теста из файла в DataFrame

In [8]:
test_excel = BASE_DIR.joinpath('data/processed/NER_TEST_LD.xlsx')

df_test = pd.read_excel(test_excel, engine='openpyxl')[['ID', 'TEXT', 'ORG_CORRECT', 'ORG_DP', 'ORG_NATASHA', 'LD_NER']]

Просматриваем данные (первые n элементов)

In [9]:
# только для ускорения тестирования
df_test = df_test.head(1000)

In [10]:
for i, text in enumerate(df_test['TEXT'].values[:50]):
    _ = nlp_ner.get_spans(text)
#     nlp_ner.get_span_coordinates()
    print(f'ex {i}:')
    show_box_markup(text, nlp_ner.spans_coord, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))

ex 0:


ex 1:


ex 2:


ex 3:


ex 4:


...

ex 45:


ex 46:


ex 47:


ex 48:


ex 49:


In [15]:
for i, v in enumerate(df_test.values[:2]):
    text = v[1]
    dp = ast.literal_eval(v[3])
    nt = ast.literal_eval(v[4])
    ld = ast.literal_eval(v[5])
    display(HTML(f'<H3>ex {v[0]}:</H3>'))
    display(HTML(f'<H4>DP:</H4>'))
    show_box_markup(text, nlp_ner.get_span_coordinates_any(dp, text, False), palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))
    display(HTML(f'<H4>NT:</H4>'))
    show_box_markup(text, nlp_ner.get_span_coordinates_any(nt, text), palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))
    display(HTML(f'<H4>LD:</H4>'))
    show_box_markup(text, nlp_ner.get_span_coordinates_any(ld, text), palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))

### Подсчитываем статистику по исследуемым моделям

Запуск нашей модели на все 20 000 текстов

Внимание! Обработка длится около 30 мин.

In [11]:
%%time
# обновление поля в датафрейме

df_test['LD_NER_PIPE'] = df_test['TEXT'].apply(nlp_ner.get_spans)
df_test['LD_NER_PIPE'] = df_test['LD_NER_PIPE'].apply(str)
# df_test.to_excel(BASE_DIR.joinpath('data/interim/NER_TEST_LD.xlsx'), index=False, engine='openpyxl')

CPU times: user 5min 58s, sys: 631 ms, total: 5min 59s
Wall time: 1min 30s


In [74]:
df_test[['TEXT']].head(5).values

array([['Инвестор МУП МП ГОРОДСКОГО ОКРУГА САМАРА КРАСНОГЛИНСКИЕ БАНИ Адрес филиала банка Брянская область, город Новозыбков Банк Инвестор ИНН/КПП 100601303468 / 632501001 тел. +7-830-199-8031 корр. счет 30310411361955797442'],
       ['Поставщик ОО НИЖЕГОРОДСКАЯ РЕГИОНАЛЬНАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ ПРАВОВОЙ ПОДДЕРЖКИ БЫСТРЫЙ ЛЕГИТИМНЫЙ АРБИТРАЖ СПОРОВ ИНН КПП Субподрядчик 601500724 / 611501001 тел. +8-( 845) -147-5726 счет № 42160059899708393439 Филиал банка З-чик КОММЕРЧЕСКИЙ БАНК "КОМПАНИЯ РОЗНИЧНОГО КРЕДИТОВАНИЯ" (ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО) 183116686028 / 163201001 БИК 040557259 Адрес банка Подрядчик Чувашская республика'],
       ['Субподрядчик СЕВЕРНОЕ ЛИНЕЙНОЕ ПРОИЗВОДСТВЕННОЕ УПРАВЛЕНИЕ МАГИСТРАЛЬНЫХ ГАЗОПРОВОДОВ (СЕВЕРНОЕ ЛПУМГ) 631226435882 ГОРОД МОСКВА УЛИЦА УСАЧЁВА ДОМ 24 Филиал банка Плательщик ООО Банк "Саратов" тел. 8 344 741 9083 БИК 044175088'],
       ['Инвестор ООО ДЮКА ИНН КПП 564500394 / 231201001 624134, Свердловская обл, г Новоуральск, ул Чурина, д 12/2, оф 

In [12]:
# подсчет статистики
org_text = df_test['TEXT']
correct_result = df_test['ORG_CORRECT']
dp_result = df_test['ORG_DP']
nt_result = df_test['ORG_NATASHA']
ld_result = df_test['LD_NER_PIPE']

correct_result = nomalize_spans(correct_result)
dp_result = nomalize_spans(dp_result, False)
nt_result = nomalize_spans(nt_result)
ld_result = nomalize_spans(ld_result)

res = [get_stat(df_test, correct_result, ld_result, 'LD_NER'),
       get_stat(df_test, correct_result, dp_result, 'DPAVLOV_NER'),
       get_stat(df_test, correct_result, nt_result, 'NATASHA_NER')]
df_res = pd.DataFrame(res)[['name','correct', 'incorrect', 'partial', 'missed', 'spurious', 'possible', 'actual', 'precision', 'recall', 'f1', 'acuracy']]

In [13]:
df_res

,name,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1,acuracy
0,LD_NER,817,30,0,7,29,854,876,0.932648,0.956674,0.944509,0.944859
1,DPAVLOV_NER,445,227,0,233,671,905,1343,0.331348,0.491713,0.395907,0.219989
2,NATASHA_NER,113,380,0,362,355,855,848,0.133255,0.132164,0.132707,0.151637


In [57]:
# 'models/ner_dpbert_3epoch_95f1ORG_99acc.bin' = 100k
df_res

,name,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1,acuracy
0,LD_NER,16662,488,0,81,383,17231,17533,0.950322,0.966978,0.958578,0.958861
1,DPAVLOV_NER,8800,4660,0,4655,13690,18115,27150,0.324125,0.485785,0.388821,0.209639
2,NATASHA_NER,2219,7814,0,7236,7582,17269,17615,0.125972,0.128496,0.127222,0.130189


In [25]:
#ner_dpbert_37epoch_86f1_93acc.bin
df_res

,name,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1,acuracy
0,LD_NER,12478,4334,0,460,2692,17272,19504,0.639766,0.722441,0.678595,0.662315
1,DPAVLOV_NER,8800,4660,0,4655,13690,18115,27150,0.324125,0.485785,0.388821,0.209639
2,NATASHA_NER,2219,7814,0,7236,7582,17269,17615,0.125972,0.128496,0.127222,0.130189


In [16]:
#с учетом обновленной постобработки вывода НЕРа и порогом меры уверенности 0.8
df_res

,name,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1,acuracy
0,LD_NER,12432,4261,0,556,2693,17249,19386,0.641288,0.720737,0.678695,0.663715
1,DPAVLOV_NER,8800,4660,0,4655,13690,18115,27150,0.324125,0.485785,0.388821,0.209639
2,NATASHA_NER,2219,7814,0,7236,7582,17269,17615,0.125972,0.128496,0.127222,0.130189


In [29]:
# До обновления постобработки с учетм мер уверенности
df_res

,name,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1,acuracy
0,LD_NER,12198,4517,0,538,3959,17253,20674,0.590016,0.707007,0.643236,0.613433
1,DPAVLOV_NER,8800,4660,0,4655,13690,18115,27150,0.324125,0.485785,0.388821,0.209639
2,NATASHA_NER,2219,7814,0,7236,7582,17269,17615,0.125972,0.128496,0.127222,0.130189


### Подсчитываем статистику по шаблонам, с перепутанным порядком сущностей = из тестов на обучении

In [13]:
import ast
test_excel = BASE_DIR.joinpath('data/interim/df_test.xlsx')

df_test_rnd = pd.read_excel(test_excel, engine='openpyxl')
df_test_rnd['TEXT'] = df_test_rnd['tokens'].apply(lambda x: ' '.join(ast.literal_eval(x)))
df_test_rnd['ORG_CORRECT'] = df_test_rnd['spans'].apply(lambda x: [i.split(':')for i in ast.literal_eval(x)])
df_test_rnd['ORG_CORRECT'] = df_test_rnd['ORG_CORRECT'].apply(str)

In [14]:
for i, text in enumerate(df_test_rnd['TEXT'].values[50:100]):
    _ = nlp_ner.get_spans(text)
#     nlp_ner.get_span_coordinates()
    print(f'ex {i}:')
    show_box_markup(text, nlp_ner.spans_coord, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))

ex 0:


ex 1:


ex 2:


ex 3:


ex 4:


...

ex 45:


ex 46:


ex 47:


ex 48:


ex 49:


In [50]:
df_test_rnd['TEXT'].values[4]

'Исполнитель Индивидуальный предприниматель Ефимова Наталья Ивановна , +8-826-568-7012 Адрес банка Исп-тель 190098 , г Санкт-Петербург , Адмиралтейский р-н , пл Труда , д 4 литер в , пом 212 231111056 Небанковская кредитная организация " МОНЕТА " ( общество с ограниченной ответственностью ) рас/сч 40253440786260473970 Адрес банка 614015 , Пермский край , г Пермь , Ленинский р-н , ул Екатерининская , д 18 тел. 8-885-63-06-595 кор/сч 30647910558312445795 БИК 046792177'

In [16]:
%%time
df_test_rnd['LD_NER_PIPE'] = df_test_rnd['TEXT'].apply(nlp_ner.get_spans)
df_test_rnd['LD_NER_PIPE'] = df_test_rnd['LD_NER_PIPE'].apply(str)

CPU times: user 6min 4s, sys: 710 ms, total: 6min 5s
Wall time: 1min 32s


In [17]:
df_test_rnd

,spans,tokens,TEXT,ORG_CORRECT,LD_NER_PIPE
0,"['STAT: 8 ( 935 ) 855 1436', 'LOC: 115162 , г ...","['8', '(', '935', ')', '855', '1436', 'Адрес',...","8 ( 935 ) 855 1436 Адрес Зак-чик 115162 , г Мо...","[['STAT', ' 8 ( 935 ) 855 1436'], ['LOC', ' 11...","[['STAT', '8 ( 935 ) 855 1436'], ['O', 'Адрес ..."
1,"['INNKPP: 6119008174', 'LOC: РЕСПУБЛИКА ТАТАРС...","['ИНН/КПП', '6119008174', 'тел.', 'РЕСПУБЛИКА'...",ИНН/КПП 6119008174 тел. РЕСПУБЛИКА ТАТАРСТАН Г...,"[['INNKPP', ' 6119008174'], ['LOC', ' РЕСПУБЛИ...","[['O', 'ИНН/КПП'], ['INNKPP', '6119008174'], [..."
2,"['ORG: ИП', 'PER: Яковлева О. А. ,.', 'PER: Гр...","['ИП', 'Яковлева', 'О.', 'А.', ',.', 'именуемо...","ИП Яковлева О. А. ,. именуемое в в лице замест...","[['ORG', ' ИП'], ['PER', ' Яковлева О. А. ,.']...","[['ORG', 'ИП Яковлева О. А'], ['O', 'именуемое..."
3,"['LOC: 410012 , Саратовская обл , г Саратов , ...","['Адрес', 'филиала', 'банка', 'Подрядчик', '41...","Адрес филиала банка Подрядчик 410012 , Саратов...","[['LOC', ' 410012 , Саратовская обл , г Сарато...","[['O', 'Адрес филиала банка Подрядчик'], ['LOC..."
4,"['PER: Ефимова Наталья Ивановна ,', 'STAT: +8-...","['Исполнитель', 'Индивидуальный', 'предпринима...",Исполнитель Индивидуальный предприниматель Ефи...,"[['PER', ' Ефимова Наталья Ивановна ,'], ['STA...","[['O', 'Исполнитель'], ['ORG', 'Индивидуальный..."
...,...,...,...,...,...
995,"['ORG: ОХРАННАЯ ОРГАНИЗАЦИЯ МИТАЗ ООО', 'INNKP...","['ОХРАННАЯ', 'ОРГАНИЗАЦИЯ', 'МИТАЗ', 'ООО', 'П...",ОХРАННАЯ ОРГАНИЗАЦИЯ МИТАЗ ООО Покупатель ИНН ...,"[['ORG', ' ОХРАННАЯ ОРГАНИЗАЦИЯ МИТАЗ ООО'], [...","[['ORG', 'ОХРАННАЯ ОРГАНИЗАЦИЯ МИТАЗ ООО'], ['..."
996,"['ORG: Индивидуальный предприниматель', 'PER: ...","['Субподрядчик', 'Индивидуальный', 'предприним...",Субподрядчик Индивидуальный предприниматель Че...,"[['ORG', ' Индивидуальный предприниматель'], [...","[['O', 'Субподрядчик'], ['ORG', 'Индивидуальны..."
997,['ORG: ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬ...,"['Исп-тель', 'ОБЩЕСТВО', 'С', 'ОГРАНИЧЕННОЙ', ...",Исп-тель ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОС...,"[['ORG', ' ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕНН...","[['O', 'Исп-тель'], ['ORG', 'ОБЩЕСТВО С ОГРАНИ..."
998,['LOC: КРАЙ КРАСНОДАРСКИЙ РАЙОН ТИМАШЕВСКИЙ ГО...,"['Исполнитель', 'Адрес', 'Клиент', 'КРАЙ', 'КР...",Исполнитель Адрес Клиент КРАЙ КРАСНОДАРСКИЙ РА...,"[['LOC', ' КРАЙ КРАСНОДАРСКИЙ РАЙОН ТИМАШЕВСКИ...","[['O', 'Исполнитель Адрес Клиент'], ['LOC', 'К..."


In [19]:
# подсчет статистики
org_text = df_test_rnd['TEXT']
correct_result = df_test_rnd['ORG_CORRECT']
ld_result = df_test_rnd['LD_NER_PIPE']

correct_result = nomalize_spans(correct_result)
ld_result = nomalize_spans(ld_result)

res = [get_stat(df_test_rnd, correct_result, ld_result, 'LD_NER')]
df_res = pd.DataFrame(res)[['name','correct', 'incorrect', 'partial', 'missed', 'spurious', 'possible', 'actual', 'precision', 'recall', 'f1', 'acuracy']]



In [23]:
correct_result

[[['агентствофурорартооо'],
  ['общественнаяорганизацияпервичнаяпрофсоюзнаяорганизацияаосевастопольскииморскоибанкобщественноиобщероссиискоиорганизациироссиискиипрофессиональныисоюзработниковсудостроения']],
 [['ооосвязьинформ'],
  ['публичноеакционерноеобществокоммерческиибанксаммитбанк']],
 [['ип'], ['яковлеваоа'], ['грефгерманоскарович']],
 [['акционерныикоммерческиибанкприморьепубличноеакционерноеобщество']],
 [['ефимованатальяивановна'],
  ['небанковскаякредитнаяорганизациямонетаобществосограниченноиответственностью']],
 [['обществосограниченноиответственностьюцентрвзаимныхинвестиции']],
 [['ип'], ['равчеевевгенииюрьевич'], ['кброспромбанкооо']],
 [['дилбанкобществосограниченноиответственностью']],
 [['акционерноеобществоросгеология'],
  ['акционерноеобществосевастопольскииморскоибанк']],
 [['панфиловнп']],
 [['ип'], ['турищевев'], ['шаталовмм']],
 [],
 [['миндзяксс']],
 [['ооовивалаб'], ['филиалкбевротрастзаогастрахань']],
 [['индивидуальныипредприниматель'], ['петрищеваларисаюрь

### Поднимаем модельку с диска

In [10]:
# поднимаем модельку из контрольной точки

model_path = 'models/ner_dpbert_3epoch_95f1ORG_99acc.bin'
# model_path = 'models/ner_dpbert_20epoch_93f1_97acc.bin/checkpoint-625000'

model_checkpoint = BASE_DIR.joinpath(model_path)



model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [13]:
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-INNKPP', 'I-INNKPP', 'B-RSKS', 'I-RSKS', 'B-STAT', 'I-STAT']


tokens = tokenizer(text, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}

with torch.no_grad():
    pred = model(**tokens)
# pred.logits.shape


indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
for t, idx in zip(token_text, indices):
    print(f'{t:15s} {label_list[idx]:10s}')

[CLS]           B-INNKPP  
поставщик       O         
о               B-ORG     
##о             B-ORG     
нижегород       I-ORG     
##ская          I-ORG     
региональная    I-ORG     
общественная    I-ORG     
организация     I-ORG     
право           I-ORG     
##во            I-ORG     
##и             I-ORG     
поддержки       I-ORG     
быстр           I-ORG     
##ы             I-ORG     
##и             I-ORG     
легитим         I-ORG     
##ны            I-ORG     
##и             I-ORG     
арбитраж        I-ORG     
споров          O         
инн             O         
к               O         
##пп            O         
субподряд       O         
##чик           O         
601             B-INNKPP  
##500           B-INNKPP  
##72            B-INNKPP  
##4             B-INNKPP  
/               O         
611             B-INNKPP  
##501           B-INNKPP  
##001           B-INNKPP  
тел             O         
.               O         
+               B-STAT    
8

### Проверяем работоспособность модели

In [17]:
texts = [
        'Ромашка-профиль, ООО  кор. счет 30101810300000000709 9204000492/92040001  "ЛИТЛТОН",ООО 299011 '\
        'г. Севастополь ул. Адмирала Октябрьского д. 13 офис 1 тел (0692) 671710 ‚  в '\
        'Банк вфыSSF d АО "ТААТТА", г. Иркутск БИК 049805709 40702810500000007563   по ОКПО',
        'валюта наименование код Российская рубль 643',
        'Инвестор ООО ДЮКА ИНН КПП 564500394 / 231201001 624134, Свердловская обл, г Новоуральск, ул Чурина, д 12/2, оф 20 рс 44468253734659494408 Филиал банка П-щик АКЦИОНЕРНОЕ ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК "КУБАНСКИЙ ТОРГОВЫЙ БАНК" ',
        'СЕВЕРНОЕ ЛИНЕЙНОЕ ПРОИЗВОДСТВЕННОЕ УПРАВЛЕНИЕ МАГИСТРАЛЬНЫХ ГАЗОПРОВОДОВ (СЕВЕРНОЕ ЛПУМГ) 631226435882 ГОРОД МОСКВА УЛИЦА УСАЧЁВА ДОМ 24 Филиал банка Плательщик ООО Банк "Саратов" тел. 8 344 741 9083 БИК ',
        'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "МАКТЕЛ" ИНН П-щик 6330058874',
        'исполнитель МКУ ЕДДС Кемеровская область, город Кемерово счет № 43209075037153306310 банк ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО "ВОСТОККРЕДИТБАНК" к/с 30732437717823862544 тел. +7 (820) 529 92 19',
        'ООО АНТ-СОФТ ИНН/КПП 3911800906 / 431201001 Банк ИНН КПП 5519504954 / тел. +8-( 873) -958-9191 БИК 042306734',
        'З-чик АО ПРОИЗВОДСТВЕННАЯ КОМПАНИЯ СТАНКОПРЕСС ИНН Покупатель 770856524 ОБЛАСТЬ АМУРСКАЯ ГОРОД БЛАГОВЕЩЕНСК УЛИЦА ЛЕНИНА 181 Банк Подрядчик КБ "Русский ипотечный банк" (ООО) тел. +7 (982) 099 40 08 045402896',
        'МУП МП ГОРОДСКОГО ОКРУГА САМАРА КРАСНОГЛИНСКИЕ БАНИ +7-830-199-8031 Адрес филиала банка Брянская область, город Новозыбков Банк Инвестор ИНН/КПП 100601303468 / 632501001 корр. счет 30310411361955797442',
        ]

## Под капотом

### Получаем теги из вывода модели

In [18]:


text = 'Инвестор МУП МП ГОРОДСКОГО ОКРУГА САМАРА КРАСНОГЛИНСКИЕ БАНИ Адрес филиала банка Брянская область, город Новозыбков Банк Инвестор ИНН/КПП 100601303468 / 632501001 тел. +7-830-199-8031 корр. счет 30310411361955797442'


# получаем результаты от модельки
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-INNKPP', 'I-INNKPP', 'B-RSKS', 'I-RSKS', 'B-STAT', 'I-STAT']

print(text)
tokens = tokenizer(text, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}

with torch.no_grad():
    pred = model(**tokens)

indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
# print(indices)
# переводим результаты модельки в финальные теги
tokens = NerTokes(token_text, indices, label_list)

# получаем пары: слово-токены
spans = tokens.join_tokens()
# выбираем нужный токен для слова
spans = tokens.normalize_tokens()

# объединяем I-токены
tokens.join_i_tag()
# объединяем BI-токены
tokens.join_bi_tag()
# финальные теги
spans = tokens.get_final_tags()


df = pd.DataFrame(spans, columns=['Тег', 'Сущность'])

display(HTML(df[['Сущность', 'Тег']].to_html()))

Инвестор МУП МП ГОРОДСКОГО ОКРУГА САМАРА КРАСНОГЛИНСКИЕ БАНИ Адрес филиала банка Брянская область, город Новозыбков Банк Инвестор ИНН/КПП 100601303468 / 632501001 тел. +7-830-199-8031 корр. счет 30310411361955797442


,Сущность,Тег
0,инвестор,O
1,муп мп городского округа самара красноглинские,ORG
2,бани,LOC
3,адрес филиала банка,O
4,"брянская область, город новозыбков",LOC
5,банк инвестор инн / кпп,O
6,100601303468,INNKPP
7,/,O
8,632501001,INNKPP
9,тел.,O


In [71]:
token_text_ = ['[CLS]',]
indices_ = [0]
pos = [(-1,-1)]


for r in nlp_res:
    token_text_.append(r['word'])
    indices_.append(int(r['entity'].split('_')[-1]))
    pos.append((r['start'], r['end']))
    print(r)
    
    
token_text_.append('[SEP]')
indices_.append(0)
pos.append((-1,-1))

NameError: name 'nlp_res' is not defined

In [102]:
scores = ['-1', 0.9919736385345459, 0.5316126942634583, 0.892160952091217, 0.722357988357544, 0.47433948516845703, 0.5830104351043701, 0.6114307045936584, 0.6053929328918457, 0.641360878944397, 0.7186465859413147, 0.9984973669052124, 0.9993856549263, 0.9989122152328491, 0.9997438788414001, 0.986339807510376, 0.9700685739517212, 0.9955838322639465, 0.9995989203453064, 0.9998665452003479, 0.9999608397483826, 0.6930971741676331, 0.5729564428329468, 0.4943584203720093, 0.8895936012268066, 0.8399950265884399, 0.7741530537605286, 0.9995461106300354, 0.9993852376937866, 0.9993306994438171, 0.9994500279426575, 0.8683413863182068, 0.7902174592018127, 0.9883625507354736, 0.9969802498817444, 0.9994044899940491, 0.9985039830207825, 0.901045560836792, 0.990676999092102, 0.9699495434761047, 0.9867011308670044, 0.9893844127655029, 0.9975012540817261, 0.9900829792022705, 0.9992552399635315, 0.9979585409164429, 0.996084988117218, 0.9918866157531738, 0.9902769923210144, 0.9987055659294128, 0.997890830039978, 0.9774914383888245, 0.5286502242088318, 0.6814475655555725, 0.9188513159751892, 0.9091649651527405, 0.9992742538452148, 0.9990254044532776, 0.9953714609146118, 0.9992408156394958, 0.9985008835792542, 0.9965887069702148, 0.9980633854866028, 0.9998336434364319, 0.9998230338096619, 0.9999509453773499, 0.9999952912330627, 0.9995056986808777, 0.998753547668457, 0.9994767308235168, 0.9999637603759766, 0.9999855756759644, 0.9995564818382263, 0.9993045330047607, 0.9999819397926331, 0.9999921321868896, 0.9999508857727051, 0.9223501086235046, 0.6846923232078552, 0.9747717976570129, 0.745853841304779, 0.7317254543304443, 0.9069281816482544, 0.9970802068710327, 0.9982325434684753, 0.9950548410415649, 0.990630030632019, 0.9905487895011902, 0.995449423789978, 0.991654634475708, 0.9420721530914307, 0.9198526740074158, 0.9682608246803284, 0.996670663356781, 0.9519242644309998, 0.778010368347168, 0.6670175790786743, 0.5139511227607727, 0.822007954120636, 0.7715737819671631, 0.9972951412200928, 0.9951765537261963, 0.999193012714386, 0.9947410821914673, 0.9959551095962524, 0.9999322295188904, 0.9988448023796082, 0.9690274596214294, '-1']
positions = [(-1, -1), (0, 9), (10, 12), (13, 16), (16, 19), (20, 22), (22, 25), (26, 28), (28, 33), (34, 37), (38, 46), (47, 50), (50, 52), (52, 54), (54, 56), (57, 58), (58, 60), (61, 70), (71, 75), (75, 77), (77, 80), (81, 85), (86, 89), (89, 90), (90, 92), (92, 93), (94, 102), (103, 104), (105, 116), (116, 117), (118, 134), (135, 146), (146, 147), (148, 152), (153, 154), (154, 162), (162, 163), (164, 167), (167, 168), (168, 169), (169, 171), (172, 175), (175, 178), (178, 180), (180, 182), (183, 184), (185, 188), (188, 191), (191, 194), (195, 200), (201, 208), (209, 214), (215, 218), (218, 219), (219, 221), (221, 222), (223, 226), (226, 228), (228, 229), (229, 230), (231, 235), (235, 238), (238, 239), (240, 243), (243, 244), (244, 245), (246, 249), (250, 255), (255, 257), (257, 258), (258, 259), (260, 262), (263, 266), (266, 269), (269, 270), (271, 272), (273, 274), (275, 280), (281, 282), (283, 296), (296, 299), (299, 300), (301, 305), (306, 309), (309, 311), (311, 313), (313, 315), (315, 317), (317, 319), (319, 321), (321, 323), (323, 325), (325, 326), (327, 330), (330, 331), (332, 334), (334, 335), (335, 337), (337, 338), (338, 340), (341, 344), (344, 345), (346, 347), (347, 348), (349, 352), (353, 356), (357, 359), (360, 362), (-1, -1)]

In [105]:
import numpy as np
def __get_single_tag(tags, sc):
    """
    Предварительный выбор тэга
    """
    return tags[np.argmax(sc)]

sp = ""
spans = []
tags = []
i = 0
conj = '##'
word_pos = []
scs = []
for t, idx, sc, pos in zip(token_text, indices, scores, positions):
    if t not in ['[CLS]', '[SEP]']:
        if t[:2] == conj:
            sp += t[2:]
            tags.append(label_list[idx])
            scs.append(sc)
            wp[1] = pos[1]
        else:
            sp += f' {t}'
            tags.append(label_list[idx])
            scs.append(sc)
            wp = [pos[0], pos[1]]
        if i < len(token_text) - 1:
            if token_text[i + 1][:2] != conj:
                spans.append([sp.strip(), __get_single_tag(tags, sc)])
                word_coordinates.append(wp)
                sp = ""
                sc = []
                wp = []
                tags = []
    i += 1

In [52]:
from fuzzysearch import find_near_matches
spans = []
for t in filter(lambda x: x[1] != 'O', df[['Сущность', 'Тег']].values):
    for m in find_near_matches(t[0], text.lower().replace('й', 'и'), max_l_dist=2): #, max_deletions=1, max_insertions=1
        
        if "".join(m.matched.split()).replace('й', 'и') == "".join(t[0].split()):
            spans.append((m.start, m.end, t[1]))
show_box_markup(text, spans, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))

### Как выводит модель для сравнения с нашим выводом финтальных тегов

In [18]:
from transformers import pipeline
model_path = 'models/ner_dpbert_7epoch_56f1ORG_96acc.bin'
model_path = 'models/ner_dpbert_3epoch_95f1ORG_99acc.bin'
# model_path = 'models/ner_dpbert_4epoch_99f1ORG_99acc.bin'

model_checkpoint = BASE_DIR.joinpath(model_path)



model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

nlp = pipeline('ner', model=model, tokenizer=tokenizer)

In [23]:
text = 'Ромашка, ООО ООО "Гарант-СВ" ИНН КПП 9103007830 910301001 ООО "ГАРАНТ-СВ” А ”5№. heOFVS1 Адрес 298685. РФ респ. Крым гор. Ялта ул Генерала О оякова. д. 9 рс 44486183785362839087'
text = 'Шешдиров Руслан Искандерович, ИП ‚ ИНН 164511946205'
text = 'Индивидуальный предприниматель Шешдиров Руслан Искандерович ‚ ИНН 164511946205 ООО "Гарант-СВ"'
text = 'МУП МП ГОРОДСКОГО ОКРУГА САМАРА КРАСНОГЛИНСКИЕ БАНИ +7-830-199-8031 Адрес филиала банка Брянская область, город Новозыбков Банк Инвестор ИНН/КПП 100601303468 / 632501001 корр. счет 30310411361955797442'
text = 'Исполнитель Индивидуальный предприниматель Ефимова Наталья Ивановна , +8-826-568-7012 Адрес банка Исп-тель 190098 , г Санкт-Петербург , Адмиралтейский р-н , пл Труда , д 4 литер в , пом 212 231111056 Небанковская кредитная организация " МОНЕТА " ( общество с ограниченной ответственностью ) рас/сч 40253440786260473970 Адрес банка 614015 , Пермский край , г Пермь , Ленинский р-н , ул Екатерининская , д 18 тел. 8-885-63-06-595 кор/сч 30647910558312445795 БИК 046792177'
text = 'г. село ул. д. по ООО торговый дом "Рублевский" ИНН 5047042767 121596 Москва г Горбунова ул дом № 4 корпус 1 тел. : (495) 771-69-74 (495) 771-71-14 р / с 40702810600011475167 в Банк АО ЮНИКРЕДИТ Банк БИК 044525545 к / с 30101810300000000545 по ОКПО ! 55032079'
# text = 'Инвестор ООО ДЮКА ИНН КПП 564500394 / 231201001 624134, Свердловская обл, г Новоуральск, ул Чурина, д 12/2, оф 20 рс 44468253734659494408 Филиал банка П-щик АКЦИОНЕРНОЕ ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК "КУБАНСКИЙ ТОРГОВЫЙ БАНК" '
print(nlp(text))

[{'word': 'г', 'score': 0.9358559846878052, 'entity': 'LABEL_5', 'index': 1, 'start': 0, 'end': 1}, {'word': '.', 'score': 0.8453817367553711, 'entity': 'LABEL_6', 'index': 2, 'start': 1, 'end': 2}, {'word': 'село', 'score': 0.9999122619628906, 'entity': 'LABEL_6', 'index': 3, 'start': 3, 'end': 7}, {'word': 'ул', 'score': 0.999908983707428, 'entity': 'LABEL_6', 'index': 4, 'start': 8, 'end': 10}, {'word': '.', 'score': 0.9999154806137085, 'entity': 'LABEL_6', 'index': 5, 'start': 10, 'end': 11}, {'word': 'д', 'score': 0.9998620748519897, 'entity': 'LABEL_6', 'index': 6, 'start': 12, 'end': 13}, {'word': '.', 'score': 0.9994387030601501, 'entity': 'LABEL_6', 'index': 7, 'start': 13, 'end': 14}, {'word': 'по', 'score': 0.998621940612793, 'entity': 'LABEL_6', 'index': 8, 'start': 15, 'end': 17}, {'word': 'о', 'score': 0.9993658065795898, 'entity': 'LABEL_3', 'index': 9, 'start': 18, 'end': 19}, {'word': '##оо', 'score': 0.9988380670547485, 'entity': 'LABEL_3', 'index': 10, 'start': 19, '

In [24]:
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-INNKPP', 'I-INNKPP', 'B-RSKS', 'I-RSKS', 'B-STAT', 'I-STAT']
for w in nlp(text, aggregate_words='simple'):
    index = int(w['entity'].split('_')[-1])
    w['entity'] = label_list[index]
    print(w)

{'word': 'г', 'score': 0.9358559846878052, 'entity': 'B-LOC', 'index': 1, 'start': 0, 'end': 1}
{'word': '.', 'score': 0.8453817367553711, 'entity': 'I-LOC', 'index': 2, 'start': 1, 'end': 2}
{'word': 'село', 'score': 0.9999122619628906, 'entity': 'I-LOC', 'index': 3, 'start': 3, 'end': 7}
{'word': 'ул', 'score': 0.999908983707428, 'entity': 'I-LOC', 'index': 4, 'start': 8, 'end': 10}
{'word': '.', 'score': 0.9999154806137085, 'entity': 'I-LOC', 'index': 5, 'start': 10, 'end': 11}
{'word': 'д', 'score': 0.9998620748519897, 'entity': 'I-LOC', 'index': 6, 'start': 12, 'end': 13}
{'word': '.', 'score': 0.9994387030601501, 'entity': 'I-LOC', 'index': 7, 'start': 13, 'end': 14}
{'word': 'по', 'score': 0.998621940612793, 'entity': 'I-LOC', 'index': 8, 'start': 15, 'end': 17}
{'word': 'о', 'score': 0.9993658065795898, 'entity': 'B-ORG', 'index': 9, 'start': 18, 'end': 19}
{'word': '##оо', 'score': 0.9988380670547485, 'entity': 'B-ORG', 'index': 10, 'start': 19, 'end': 21}
{'word': 'торгов', '

In [22]:
for i, v in enumerate(df_test.values[:4]):
    text = v[1]
    dp = ast.literal_eval(v[3])
    nt = ast.literal_eval(v[4])
    ld = ast.literal_eval(v[5])
    display(HTML(f'<H3>ex {v[0]}:</H3>'))
    display(HTML(f'<H4>DP:</H4>'))
    show_box_markup(text, nlp_ner.get_span_coordinates_any(dp, text, False), palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))
    display(HTML(f'<H4>NT:</H4>'))
    show_box_markup(text, nlp_ner.get_span_coordinates_any(nt, text), palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))
    display(HTML(f'<H4>LD:</H4>'))
    nlp_ner.get_spans(text)
    show_box_markup(text, nlp_ner.spans_coord, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN, INNKPP=ORANGE, STAT=PURPLE, RSKS=BROWN))